# MicaSense Processing

## Dependencies

In [ ]:
import os
import rasterio
import glob
import numpy as np

from rasterio.enums import Resampling
from dataclasses import asdict

from seadrone.data_structures import Profile, GeoreferencePartition, MergePartition, Partition, Resampling as ResamplingSeadrone
from seadrone.enums import SensorType, FlightMode
from seadrone.processing import MicaSenseProcessor as processor, get_sensor
from seadrone.raster import RasterUtils

## Variables Set Up

**project_path**: It is used to define some useful subfolders of our flight and it's the folder path that contains all the flight data.

**exiftool_path**: The full path of 'exiftool.exe' file saved in dependencies folder. It's used to read the metadata of the micasense captures.

**file_path**: Is the full path of some flight capture that contains metadata like GPS, ...; It's used to generate automatically the sensor data needed for georeferencing, ...

**flight_folder**: This folder contains all the flight captures in .tif format.

**sky_folder**: This folder contains sky captures for calibration.

**panel_folder**: This folder contains panel captures for calibration.

**align_folder**: This folder contains one flight capture for band alignment.

**flight_metadata_out_name**: The full path where the metadata.csv file of each flight captures will be saved.

**sky_metadata_out_folder**: The full path where the metadata.csv file of each sky captures will be saved.

**georeference_out_folder**: This folder contains all the georefered captures. It's structure is "georeferences -> folder -> partition -> captures"

**merge_out_folder**: This folder contains all the merged captures. It's structure is "merges -> folder -> partition -> merge_method -> captures"

**resample_out_folder**: This folder contains all the resampled captures. It's structure is "resamples -> folder -> partition -> merge_method -> captures".

**fligth_lines_out_name**: The typical name for out fligth_lines.yaml file that contains all the captures that we want to georeference, merge, ...

**sensor**: A representation of the sensor specifications of the flight like focal_length, bands_number, ...

**img_profile**: The profile needed to georefence thumbnails.

**bands_profile**: The profile needed to georefence rrs captures.

**products_profile**: The profile needed to georefence products captures like chl-a.

**img_flip_axis** & **bands_flip_axis**: For georeference it's necessary to flip the data matrix [bands, height, width]. In MicaSense case, flipping the height is sufficient for thumnails and flipping height and width for rrs and products.

**steps_in_flight_line**: When processing a flight line, sometimes we want to skip some captures. Setting this variable to 1 means no skip, just process the next capture, 2 means skip 1 capture and so on.

**partitions**: Sometimes we don't want to include all flight lines in our mosaic. This dictionary defines the some slices to process all, even and odd flight lines.

**partitions_for_georeference**: The flight lines to georeference

**partitions_for_merge**: The flight lines to merge

**partitions_for_resample**: The flight lines to resample

**merge_methods**: The merge method to use. Available options are 'mean', 'firsr', 'min' and 'max'.

**resampling_methods**: Sometime we want to redude the spatial resolution of our mosaic. To achieve that we define this variable with the structure: 

**resampling_methods**: Sometime we want to redude the spatial resolution of our mosaic. To achieve that we define a Resampling variable contained in the **data_structures.py** file that receives: The resampling method, the x scale, the y scale and a boolean indicating whether to downsample or upsample.

**correction**: We normally have to reduce the flight angle by -90 o -270 because of the cameratransform library.

**altitude**: Average altitude of the drone during the flight. We usually set this to None. Meaning that we'll use the altitude of each capture for the georeferencing part.

**yaw**: Flight angle.

**pitch**: Average pitch of the drone during the flight. We usually set this to 0.

**roll**: Average roll of the drone during the flight. We usually set this to 0.

**band_names**: If we set this variable using sensor.band_names or whatever we want, the mosaic will be splitted if the number of bands is the mosaic has the same length as band_names. In other case, the mosaic will not be splitted. The variable have the structure List[str]. Example: ['blue', 'red', ...]

In [ ]:
project_path = r'C:\Users\sergi\Desktop\Dummy\MicaSense' # editable
exiftool_path = r'C:\Users\sergi\Documents\repos\seadronelib\dependencies\exiftool.exe' # editable
file_path = r'C:\Users\sergi\Desktop\Dummy\MicaSense\align_img\IMG_0003_1.tif' # editable

flight_folder = os.path.join(project_path, 'raw_water_imgs')
sky_folder = os.path.join(project_path, 'raw_sky_imgs')
panel_folder = os.path.join(project_path, 'panel')
align_folder = os.path.join(project_path, 'align_img')

flight_metadata_out_name = os.path.join(project_path, 'metadata', 'water_imgs_metadata.csv')
sky_metadata_out_name = os.path.join(project_path, 'metadata', 'sky_imgs_metadata.csv')

georeference_out_folder = os.path.join(project_path, 'georeferences')
merge_out_folder = os.path.join(project_path, 'merges')
resample_out_folder = os.path.join(project_path, 'resamples')
fligth_lines_out_name = os.path.join(project_path, 'flight_lines.yaml')

sensor = get_sensor(SensorType.MICASENSE, file_path, exiftool_path = exiftool_path)

img_profile =  Profile(dtype = rasterio.uint8, count = 3, height = sensor.height, width = sensor.width, nodata = 0) # editable
bands_profile =  Profile(dtype = rasterio.float32, count = sensor.bands_number, height = sensor.height, width = sensor.width, nodata = np.NAN) # editable
products_profile =  Profile(dtype = rasterio.float32, count = 1, height = sensor.height, width = sensor.width, nodata = np.NAN) # editable

img_flip_axis = [1] # editable
bands_flip_axis = [1, 2] # editable

steps_in_flight_line = 2 # editable

partitions = { # editable
    'even' : Partition('even', 0, None, 2),
    'odd' : Partition('odd', 1, None, 2),
    'all' : Partition('all', 0, None, 1),
}

partitions_for_georeference = ['even'] # editable
partitions_for_merge = ['even'] # editable
partitions_for_resample = ['even'] # editable
merge_methods = ['mean'] # editable

resampling_methods = [ # editable
    ResamplingSeadrone(Resampling.average, 15, 15, True),
    ResamplingSeadrone(Resampling.average, 30, 30, True),
    ResamplingSeadrone(Resampling.average, 50, 50, True),
]


# ## Processing
# ## Metadata
# #### Flight
flight_metadata = processor.get_captures_metadata(flight_folder)
processor.save_metadata(flight_metadata, flight_metadata_out_name)


# #### Sky
sky_metadata = processor.get_captures_metadata(sky_folder)
processor.save_metadata(sky_metadata, sky_metadata_out_name)


# #### Load metadata
flight_metadata = None
sky_metadata = None

if os.path.exists(flight_metadata_out_name):
    in_folder = os.path.dirname(flight_metadata_out_name)
    in_name = os.path.basename(flight_metadata_out_name)
    flight_metadata = processor.load_metadata(in_folder, in_name)

if os.path.exists(sky_metadata_out_name):
    in_folder = os.path.dirname(sky_metadata_out_name)
    in_name = os.path.basename(sky_metadata_out_name)
    sky_metadata = processor.load_metadata(in_folder, in_name)


# ## Flight Lines
# #### Extract Flight Lines
correction = -90 # editable
altitude = None # editable
yaw = 90 # editable
pitch = 0 # editable
roll = 0 # editable

fligth_lines = processor.compute_flight_lines(flight_metadata.Yaw, altitude, yaw + correction, pitch, roll, FlightMode.OVERLAP_FIXED)
processor.save_flight_lines(fligth_lines, fligth_lines_out_name)


# #### Load Flight Lines
fligth_lines = None

if os.path.exists(fligth_lines_out_name):
    in_folder = os.path.dirname(fligth_lines_out_name)
    in_name = os.path.basename(fligth_lines_out_name)
    fligth_lines = processor.load_flight_lines(in_folder, in_name)


# ## Rrs
# #### Georeference
folders_to_georeference = glob.glob(os.path.join(project_path, 'rrs', '*'))

for folder_to_georeference in folders_to_georeference:
    current_georeference_out_folder = os.path.join(georeference_out_folder, os.path.basename(folder_to_georeference))
    for partition_to_use in partitions_for_georeference:
        georefence_partition = GeoreferencePartition(**asdict(partitions[partition_to_use]), profile = bands_profile)
        processor.georefence_bands(metadata = flight_metadata, partition = georefence_partition, flight_lines = fligth_lines, 
                                   in_folder = folder_to_georeference, out_folder = current_georeference_out_folder, 
                                   flip_axis = bands_flip_axis, overwrite = True)


# #### Split Bands
folders_to_georeference = glob.glob(os.path.join(project_path, 'rrs', '*'))

for folder_to_georeference in folders_to_georeference:
    current_georeference_out_folder = os.path.join(georeference_out_folder, os.path.basename(folder_to_georeference))
    for partition_to_use in partitions_for_georeference:
        rasters = glob.glob(os.path.join(current_georeference_out_folder, partition_to_use, '*'))
        rasters = [raster for raster in rasters if os.path.isfile(raster)]
        RasterUtils.split_bands_batch(raster_paths = rasters, band_names = sensor.band_names)    


# ## Thumbnails
# #### Georeference
folders_to_georeference = [os.path.join(project_path, 'thumbnails_water_imgs')]

for folder_to_georeference in folders_to_georeference:
    current_georeference_out_folder = os.path.join(georeference_out_folder, os.path.basename(folder_to_georeference))
    for partition_to_use in partitions_for_georeference:
        georefence_partition = GeoreferencePartition(**asdict(partitions[partition_to_use]), profile = img_profile)
        processor.georefence_images(metadata = flight_metadata, partition = georefence_partition, flight_lines = fligth_lines,
                                    in_folder = folder_to_georeference, out_folder = current_georeference_out_folder,
                                    flip_axis = img_flip_axis, overwrite = True)

# ## Products
# #### Georeference
folders_to_georeference = glob.glob(os.path.join(project_path, 'products', '*'))

for folder_to_georeference in folders_to_georeference:
    current_georeference_out_folder = os.path.join(georeference_out_folder, os.path.basename(folder_to_georeference))
    for partition_to_use in partitions_for_georeference:
        georefence_partition = GeoreferencePartition(**asdict(partitions[partition_to_use]), profile = products_profile)
        processor.georefence_bands(metadata = flight_metadata, partition = georefence_partition, flight_lines = fligth_lines,
                                   in_folder = folder_to_georeference, out_folder = current_georeference_out_folder,
                                    flip_axis = bands_flip_axis, overwrite = True)


# ## Merge
folders_to_merge = glob.glob(os.path.join(georeference_out_folder, '*'))
band_names = sensor.band_names

for folder_to_merge in folders_to_merge:
    current_merge_out_folder = os.path.join(merge_out_folder, os.path.basename(folder_to_merge))

    for partition_to_use in partitions_for_merge:
        for merge_method in merge_methods:
            merge_partition = MergePartition(**asdict(partitions[partition_to_use]), skip = steps_in_flight_line)
            processor.merge(metadata = flight_metadata, partition = merge_partition, flight_lines = fligth_lines,
                            in_folder = folder_to_merge, out_folder = current_merge_out_folder, method = merge_method,
                            band_names = band_names)


# ## Resample
folders_to_resample = glob.glob(os.path.join(merge_out_folder, '*'))

for folder_to_resample in folders_to_resample:
    processor.resample(folder_to_resample, resample_out_folder, resampling_methods)